In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json
from io import StringIO
import io
import shapely

DEVELOPE = True

#### weather data - Local Climatological Data (LCD)

In [2]:
df = pd.read_csv('weather130101-171130.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,11,14,15,20,23,25,40,44,45,67,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print ("Shape of data\n{}".format(df.shape))

Shape of data
(55996, 90)


In [4]:
df.drop(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 
              'LONGITUDE', 'REPORTTPYE', 'HOURLYSKYCONDITIONS', 
              'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 
              'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 
              'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 
              'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 
              'HOURLYRelativeHumidity', 'HOURLYWindDirection',
              'HOURLYPrecip', 'MonthlyMaximumTemp',
       'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH',
       'MonthlyDewpointTemp', 'MonthlyWetBulbTemp',
       'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays',
       'MonthlyStationPressure', 'MonthlySeaLevelPressure',
       'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall',
       'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp',
       'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip',
       'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip',
       'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall',
       'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth',
       'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp',
       'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp',
       'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip',
       'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow',
       'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate',
       'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue',
       'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime',
       'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays',
       'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD',
       'MonthlyTotalSeasonToDateHeatingDD',
       'MonthlyTotalSeasonToDateCoolingDD','DAILYAverageStationPressure',
       'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed',
       'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed',
       'DAILYSustainedWindDirection','DAILYMaximumDryBulbTemp',
       'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
       'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity',
       'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp',
       'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays'], axis=1, inplace=True)

In [5]:
df.columns

Index(['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'HOURLYPressureTendency',
       'HOURLYPressureChange', 'HOURLYSeaLevelPressure',
       'HOURLYAltimeterSetting', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth'],
      dtype='object')

In [6]:
df.count()

DATE                      55996
HOURLYWindSpeed           50749
HOURLYWindGustSpeed        9362
HOURLYStationPressure     53604
HOURLYPressureTendency    14083
HOURLYPressureChange       2423
HOURLYSeaLevelPressure    42318
HOURLYAltimeterSetting    53660
DAILYSunrise              55996
DAILYSunset               55996
DAILYWeather                776
DAILYPrecip                2388
DAILYSnowfall              1794
DAILYSnowDepth             1794
dtype: int64

In [7]:
df.head()

,DATE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYAltimeterSetting,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth
0,2013-01-01 00:51,10.0,22.0,29.72,8.0,NaN,29.87,29.89,720,1639,NaN,NaN,NaN,NaN
1,2013-01-01 01:51,3.0,17.0,29.73,NaN,NaN,29.88,29.90,720,1639,NaN,NaN,NaN,NaN
2,2013-01-01 02:51,7.0,NaN,29.73,NaN,NaN,29.88,29.90,720,1639,NaN,NaN,NaN,NaN
3,2013-01-01 03:51,9.0,NaN,29.74,3.0,NaN,29.88,29.91,720,1639,NaN,NaN,NaN,NaN
4,2013-01-01 04:51,7.0,NaN,29.73,NaN,NaN,29.88,29.90,720,1639,NaN,NaN,NaN,NaN


In [8]:
df.dtypes

DATE                       object
HOURLYWindSpeed           float64
HOURLYWindGustSpeed       float64
HOURLYStationPressure      object
HOURLYPressureTendency    float64
HOURLYPressureChange      float64
HOURLYSeaLevelPressure     object
HOURLYAltimeterSetting     object
DAILYSunrise                int64
DAILYSunset                 int64
DAILYWeather               object
DAILYPrecip                object
DAILYSnowfall              object
DAILYSnowDepth             object
dtype: object

In [9]:
df.HOURLYWindSpeed.unique()

array([ 10.,   3.,   7.,   9.,   6.,   5.,   0.,  11.,  nan,   8.,  13.,
        15.,  14.,  18.,  16.,  17.,  20.,  23.,  21.,  22.,  24.,  25.,
        28.,  26.])

In [10]:
df.HOURLYStationPressure.unique()

array(['29.72', '29.73', '29.74', '29.75', '29.71', '29.69', '29.77',
       '29.79', '29.80', '29.82', '29.84', '29.86', nan, '29.87', '29.88',
       '29.91', '29.92', '29.89', '29.95', '29.96', '29.97', '29.98',
       '29.99', '30.01', '30.02', '30.03', '30.04', '29.94', '29.93',
       '29.90', '29.83', '29.85', '30.08', '30.10', '30.11', '30.09',
       '30.06', '30.05', '30.14', '30.16', '30.17', '30.19', '30.22',
       '30.23', '30.24', '30.25', '30.21', '30.26', '30.28', '30.27',
       '30.20', '30.13', '29.99s', '30.07', '30.18', '30.30', '30.32',
       '30.35', '30.34', '30.31', '30.33', '30.37', '30.38', '30.39',
       '30.36', '30.15', '30.12', '30.00', '29.78', '29.68', '29.70',
       '29.76', '30.29', '29.64', '29.61', '29.60', '29.58', '29.56',
       '29.55', '29.62', '29.81', '29.65', '29.57', '29.52', '29.49',
       '29.47', '29.44', '29.45', '29.39', '29.34', '29.31', '29.29',
       '29.24', '29.17', '29.08', '29.01', '28.95', '28.94', '28.93',
       '29.02'

In [11]:
df.HOURLYWindGustSpeed.unique()

array([ 22.,  17.,  nan,  21.,  16.,  18.,  24.,  28.,  23.,  20.,  25.,
        29.,  31.,  36.,  26.,  43.,  33.,  44.,  37.,  40.,  30.,  32.,
        34.,  38.,  46.,  39.,  41.,  45.])

In [12]:
df.DAILYSunrise.unique()

array([720, 719, 718, 717, 716, 715, 714, 713, 712, 711, 710, 709, 708,
       707, 706, 705, 704, 703, 702, 701, 700, 659, 658, 657, 655, 654,
       653, 652, 650, 649, 648, 646, 645, 644, 642, 641, 639, 638, 636,
       635, 633, 632, 630, 629, 627, 626, 624, 623, 621, 619, 618, 616,
       615, 613, 611, 610, 608, 606, 605, 603, 601, 600, 558, 556, 555,
       553, 551, 550, 548, 547, 545, 543, 542, 540, 538, 537, 535, 533,
       532, 530, 529, 527, 525, 524, 522, 521, 519, 518, 516, 514, 513,
       511, 510, 508, 507, 506, 504, 503, 501, 500, 459, 457, 456, 455,
       453, 452, 451, 450, 449, 447, 446, 445, 444, 443, 442, 441, 440,
       439, 438, 437, 436, 435, 434, 433, 432, 431, 430, 429, 428, 427,
       426, 425, 424, 448, 454, 458, 502, 505, 509, 512, 515, 517, 520,
       523, 526, 528, 531, 534, 536, 539, 541, 544, 546, 549, 552, 554,
       557, 559, 604, 607, 609, 612, 617, 620, 625, 628, 631, 634, 637,
       640, 643, 647, 651, 656])

In [13]:
df.DAILYSunset.unique()

array([1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1649, 1650,
       1651, 1652, 1653, 1654, 1655, 1656, 1657, 1659, 1700, 1701, 1702,
       1703, 1705, 1706, 1707, 1708, 1710, 1711, 1712, 1713, 1714, 1716,
       1717, 1718, 1719, 1721, 1722, 1723, 1724, 1726, 1727, 1728, 1729,
       1730, 1732, 1733, 1734, 1735, 1736, 1738, 1739, 1740, 1741, 1742,
       1743, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1753, 1754, 1755,
       1756, 1757, 1758, 1759, 1800, 1801, 1802, 1803, 1805, 1806, 1807,
       1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818,
       1819, 1820, 1821, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830,
       1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841,
       1842, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853,
       1854, 1855, 1856, 1857, 1858, 1859, 1900, 1901, 1902, 1903, 1904,
       1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915,
       1916, 1917, 1918, 1919, 1920, 1921, 1922, 19

DZ:01 - Drizzle | RA:02 - Rain | SN:03 - Snow | SG:04 - Snow Grains | IC:05 - Ice Crystals 
PL:06 - Ice Pellets | GR:07 - Hail | GS:08- Small Hail an/or Snow Pellets 
UP:09 - Unknown Precipitation | BR:1 - Mist | FG:2 - Fog | FU:3 - Smoke
VA:4 - Volcanic Ash | DU:5 - Widespread Dust | SA:6 - Sand | HZ:7 - Haze
PY:8 - Spray | PO:1 - Well developed dust/sand whirls | SQ:2 - Squalls
FC:3 - Funnel CLoud, Waterspout or Tornado | SS:4 - Sandstorm | DS:5 - Duststorm

In [14]:
df.DAILYWeather.unique()

array([nan, 'HZ:08', 'BR:13 HZ:08', 'RA:16 BR:13', 'FG:01 BR:13 HZ:08',
       'FG:01 BR:13 HZ:08 RA:16', 'RA:16 BR:13 UP:19', 'SN:18 BR:13',
       'RA:16 SN:18 BR:13 UP:19 HZ:08', 'RA:16 BR:13 HZ:08',
       'RA:16 FG:01 BR:13 HZ:08', 'RA:16 FG:01 BR:13', 'SN:18 BR:13 HZ:08',
       'RA:16 SN:18 FG:01 FZFG:22 BR:13 UP:19',
       'SN:18 FZFG:22 BR:13 HZ:08 FG:01', 'RA:16 SN:18 UP:19',
       'BR:13 UP:19', 'RA:16', 'SN:18', 'SN:18 BR:13 UP:19',
       'SN:18 FG:01 FZFG:22 BR:13', 'SN:18 FG:01 BR:13',
       'RA:16 SN:18 FZFG:22 BR:13 UP:19 HZ:08 FG:01',
       'RA:16 SN:18 BR:13 UP:19', 'BR:13', 'RA:16 HZ:08', 'RA:16 SN:18',
       'SN:18 FG:01 HZ:08', 'FZRA:17 SN:18 BR:13 UP:19 HZ:08',
       'RA:16 FZRA:17 SN:18 BR:13 UP:19', 'SN:18 FG:01 BR:13 HZ:08',
       'RA:16 FZRA:17 SN:18 FZFG:22 BR:13 UP:19 HZ:08 FG:01',
       'FZRA:17 SN:18 FZFG:22 BR:13 UP:19 HZ:08 FG:01',
       'SN:18 FZFG:22 FG:01', 'RA:16 FG:01 BR:13 UP:19 HZ:08',
       'RA:16 FG:01 BR:13 UP:19', 'RA:16 SN:18 FG:01

In [15]:
#T indicates trace amount.
df.DAILYPrecip.unique()

array([nan, '0.00', 'T', '0.55', '0.57', '0.02', '0.09', '0.12', '0.63',
       '0.69', '0.07', '0.22', '0.06', '0.04', '0.93', '0.90', '0.01',
       '0.03', '1.15', '1.52', '0.38', '0.49', '0.40', '0.15', '0.26',
       '0.27', '0.14', '1.26', '1.56', '0.44', '0.19', '0.33', '0.54',
       '0.08', '0.79', '0.71', '0.65', '0.81', '0.36', '0.20', '0.17',
       '0.05', '0.11', '1.28', '3.02', '1.76', '0.50', '0.48', '0.18',
       '1.09', '0.92', '0.10', '0.60', '0.51', '1.81', '1.91', '0.30',
       '0.29', '0.52', '0.85', '1.00', '0.87', '0.72', '0.13', '0.95',
       '4.16', '3.82', '1.38', '1.47', '1.58', '0.24', '0.84', '0.53',
       '0.23', '0.25', '0.28', '0.37', '0.31', '0.46', '0.32', '0.43',
       '1.60', '1.66', '1.17', '0.45', '2.18', '1.98', '0.73', '0.83',
       '1.45', '1.20', '0.58', '1.40', '0.42', '0.97', '1.43', '1.78',
       '1.83', '0.16', '0.35', '2.01', '0.21', '0.86', '0.34', '0.89',
       '0.59', '4.97', '4.53', '0.41', '1.54', '1.51', '0.91', '0.99',
    

In [16]:
#The depth of frozen precipitation like snow is measured in tenths (0.1) of an inch and 
#trace amounts are defined as less than half that amount (0.05 inch).
#T indicates trace amount.
df.DAILYSnowfall.unique()

array([nan, '0.0', 'T', '1.5', '0.1', '0.3', '0.4', '6.3', '5.1', '4.0',
       '3.0', '0.7', '1.4', '5.0', '8.6', '3.1', '3.3', '11.0', '0.5',
       '1.0', '0.8', '8.0', '1.2', '9.5', '1.6', '0.2', '3.6', '5.5',
       '4.3', '16.9', '0.9', '4.8', '1.8', '7.5', '4.5', '27.3', '2.5',
       '2.8', '3.2', '0.6', '7.9', '9.4', '2.1', '7.6'], dtype=object)

In [17]:
df1 = df[['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall']]
df1.head()

,DATE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall
0,2013-01-01 00:51,10.0,22.0,29.72,720,1639,NaN,NaN,NaN
1,2013-01-01 01:51,3.0,17.0,29.73,720,1639,NaN,NaN,NaN
2,2013-01-01 02:51,7.0,NaN,29.73,720,1639,NaN,NaN,NaN
3,2013-01-01 03:51,9.0,NaN,29.74,720,1639,NaN,NaN,NaN
4,2013-01-01 04:51,7.0,NaN,29.73,720,1639,NaN,NaN,NaN


#### Storm data

In [18]:
dfbad = pd.read_csv('storm_data_search_results.csv')
dfbad.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_RANGE,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER
0,421883,NEW YORK (MANHATTAN) (ZONE),,01/31/2013,0,High Wind,50.00,,0,0,...,,,,,,,,,,1
1,436900,NEW YORK (MANHATTAN) (ZONE),,02/08/2013,1400,Winter Storm,,,0,0,...,,,,,,,,,,2
2,452589,NEW YORK CO.,NEW YORK,05/11/2013,1508,Thunderstorm Wind,61.00,,0,0,...,1,SW,1,SW,NEW YORK,40.716,-74.0165,40.716,-74.0165,3
3,452588,NEW YORK CO.,NEW YORK,05/11/2013,1508,Thunderstorm Wind,65.00,,0,0,...,2,SSW,2,SSW,NEW YORK,40.7108,-74.0161,40.7108,-74.0161,4
4,459595,NEW YORK CO.,CENTRAL PARK,06/02/2013,2259,Flash Flood,,,0,0,...,2,NNE,2,NNE,CENTRAL PARK,40.8032,-73.9314,40.801,-73.9321,5


In [19]:
dfbad['EVENT_TYPE'].unique()

array(['High Wind', 'Winter Storm', 'Thunderstorm Wind', 'Flash Flood',
       'Flood', 'Excessive Heat', 'Strong Wind', 'Dense Fog',
       'Winter Weather', 'Heavy Snow', 'Blizzard'], dtype=object)

In [20]:
dfbad[dfbad['EVENT_TYPE'] == 'Flood']

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_RANGE,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER
5,467533,NEW YORK CO.,CENTRAL PARK,06/07/2013,2205,Flood,,,0,0,...,1,NNW,2,NW,CENTRAL PARK,40.7942,-73.9547,40.7939,-73.9732,6
